In [16]:
import scipy as sp
import IPython
import numpy as np 
import matplotlib.pyplot as plt


ADD_NOISE = True

sr, audio = sp.io.wavfile.read('examples/que_es_un_filtro.wav')
if audio.dtype == np.int16:
    audio = audio/32767
elif audio.dtype == np.int8:
    audio = audio/127-1

if ADD_NOISE:
    audio = audio + np.random.normal(0, 0.05, audio.shape)

audio = audio.T # fix display.Audio 
IPython.display.Audio(data=audio, rate=sr)

In [7]:
from dispeach.filtering import low_pass_filter
audio_lp = low_pass_filter(audio, 2000, 5, sr)
IPython.display.Audio(data=audio_lp, rate=sr)

In [18]:
def distort(audio: np.ndarray, a: float) -> np.ndarray:
    z = np.pi * a
    s = 1 / np.sin(z)
    b = 1 / a

    audio_distorted = np.zeros_like(audio)
    audio_distorted[audio > b] = 1
    audio_distorted[audio <= b] = np.sin(z * audio[audio <= b]) * s

    return audio_distorted

In [19]:
# first option
audio_distorted = distort(audio_lp, 2)
# plt.plot(audio_distorted[0])
IPython.display.Audio(data=audio_distorted, rate=sr)

In [20]:
def harmonic_distortion(signal, distortion_amount=50, harmonic_brightness=50):
        # Normalize parameters to a range suitable for processing
        distortion_amount = distortion_amount / 100.0  # Scale between 0 and 1
        harmonic_brightness = harmonic_brightness / 100.0  # Scale between 0 and 1

        # Apply asymmetric distortion to enhance even harmonics
        distorted_signal = signal + distortion_amount * (signal ** 2) * np.sign(signal)
        
        # Apply harmonic brightness - this acts like a higher order enhancement
        brightness_factor = (1.0 + harmonic_brightness * (signal ** 3)) * np.sign(signal)
        distorted_signal *= brightness_factor

        # Normalize to prevent clipping
        audio_distorted = distorted_signal / np.max(np.abs(distorted_signal))
        
        return audio_distorted

In [21]:
# second option
audio_distorted2 = harmonic_distortion(audio_distorted, harmonic_brightness=100)
# plt.plot(audio_distorted[0])
IPython.display.Audio(data=audio_distorted2, rate=sr)

In [12]:
# blind test: quien contento
# inma dice: "enunciado", y oye " por favor"